In [58]:
import xml
import xml.etree.ElementTree as ET
import glob
from os import path
from collections import OrderedDict, defaultdict
import plotly.express as px

In [51]:
training_mention_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/training/event_nugget/"
# training_mention_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/eval/nugget/"

mention_ann_files = sorted(glob.glob(path.join(training_mention_dir, "*.xml")))

print(len(mention_ann_files))


158


In [52]:
doc_types = defaultdict(int)
event_type_count = defaultdict(int)
event_subtype_count = defaultdict(int)
event_realis_count = defaultdict(int)

trigger_word_dict = defaultdict(int)
span_to_count = defaultdict(set)

for ann_file in mention_ann_files:
    tree = ET.parse(ann_file)
    xml_root = tree.getroot()

    doc_id = xml_root.attrib['doc_id']
    doc_source_type = xml_root.attrib['source_type']
    
    # Update doc type count
    doc_types[doc_source_type] += 1

#     print(doc_id, doc_source_type)

    for elem in xml_root.iter('event_mention'):
        event_id = elem.attrib['id']
        event_type = elem.attrib['type']
        # Update event type count
        event_type_count[event_type] += 1
        
        event_subtype = f"{event_type}.{elem.attrib['subtype']}"
        # Update event subtype count
        event_subtype_count[event_subtype] += 1
        
        event_realis = elem.attrib['realis']
        event_realis_count[event_realis] += 1
        
        trigger_elem = elem.find('trigger')
        offset = trigger_elem.attrib['offset']
        length = trigger_elem.attrib['length']
        event_phrase = trigger_elem.text
        trigger_word_dict[event_phrase] += 1
        
        # Doc ID, doc offset, and span length uniquely identify a span 
        span_to_count[(doc_id, offset, length)].add(event_id)    

In [57]:
print(doc_types)
print(event_realis_count)
print(len(event_subtype_count))
print(len(event_type_count))
print("Total events:", sum(list(event_realis_count.values())))

trigger_count_per_span = defaultdict(int)
total_events = 0
for (doc_id, offset, length), event_id_list in span_to_count.items():
    trigger_count_per_span[len(event_id_list)] += 1
    total_events += len(event_id_list)
    
print(trigger_count_per_span)
print("Total events:", total_events)

defaultdict(<class 'int'>, {'multi_post': 77, 'newswire': 81})
defaultdict(<class 'int'>, {'actual': 3599, 'generic': 1211, 'other': 1728})
38
9
Total events: 6538
defaultdict(<class 'int'>, {1: 5886, 2: 317, 3: 6})
Total events: 6538


### Coref files

In [60]:
training_coref_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/training/event_hopper/"
# training_mention_dir = "/home/shtoshni/Research/events/data/kbp_2014-2015/data/2015/eval/nugget/"

coref_ann_files = sorted(glob.glob(path.join(training_coref_dir, "*.xml")))

In [61]:
# Global counters
event_type_counter = defaultdict(int)
event_subtype_counter = defaultdict(int)
event_realis_counter = defaultdict(int)
event_phrase_counter = defaultdict(int)
coref_chain_size_counter = defaultdict(int)

for ann_file in coref_ann_files[:1]:
    tree = ET.parse(ann_file)
    xml_root = tree.getroot()
    
    for coref_elem in xml_root.iter('hopper'):
        event_type_set = set()
        event_subtype_set = set()
        event_realis_set = set()
        event_phrase_set = set()
        coref_chain_size = 0
        for event_elem in coref_elem.iter('event_mention'):
            # Get type, subtype, realis, and text
            event_type_set.add(event_elem.attrib['type'])
            event_subtype_set.add(f"{event_type}.{elem.attrib['subtype']}")
            event_realis_set.add(elem.attrib['realis'])

            trigger_elem = elem.find('trigger')
            event_phrase_set.add(trigger_elem.text)
            coref_chain_size += 1
            
        event_type_counter[len(event_type_set)] += 1
        event_subtype_counter[len(event_subtype_set)] += 1
        event_realis_counter[len(event_realis_set)] += 1
        event_phrase_counter[len(event_phrase_set)] += 1
        coref_chain_size_counter[coref_chain_size] += 1